In [4]:
#datos
!git clone https://github.com/Jakobovski/free-spoken-digit-dataset.git


import os
import numpy as np
import librosa
import matplotlib.pyplot as plt


import tensorflow as tf
from sklearn.model_selection import train_test_split

fatal: destination path 'free-spoken-digit-dataset' already exists and is not an empty directory.


In [9]:
!pip install PyWavelets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 58.5 MB/s eta 0:00:00


In [10]:
import pywt


Procesamiento de datos

Se implementó un sistema de clasificación de audio basado en transformada wavelet y redes neuronales profundas.

El modelo toma grabaciones de voz de dígitos (0 al 9) y aprende a reconocerlos usando representaciones tiempo-frecuencia.

Se usó el dataset Free Spoken Digit Dataset (FSDD) que contiene audios en formato .wav de diferentes hablantes.



Preprocesamiento:

Conversión de señales de audio a 8,000 Hz.

Aplicación de pywt.cwt() para obtener coeficientes de wavelet.

Reducción de tamaño de los coeficientes a 128x128 para normalización.

Etiquetado: Se extrajo el dígito hablado desde el nombre del archivo.

Entrenamiento del modelo:

CNN con 2 capas convolucionales + MaxPooling + capas densas.

Función de pérdida: sparse_categorical_crossentropy.

Optimización con Adam.

Evaluación: Se usó un 20% de los datos para prueba y validación.



In [11]:


data_dir = '/content/free-spoken-digit-dataset/recordings'
X = []
y = []

#espectrograma tipo wavelet
for filename in os.listdir(data_dir):
    if filename.endswith('.wav'):
        path = os.path.join(data_dir, filename)
        signal, sr = librosa.load(path, sr=8000)

        # Transformada Wavelet
        scales = np.arange(1, 128)
        coef, xs = pywt.cwt(signal, scales, 'morl', sampling_period=1/sr)
        coef = np.abs(coef)

        # Redimensionar
        coef_resized = np.resize(coef, (128, 128))

        X.append(coef_resized)
        y.append(int(filename[0]))

X = np.array(X)
y = np.array(y)
X = X[..., np.newaxis]


In [12]:

# Separar en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelo simple CNN
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(128,128,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrenamiento
model.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 61s 774ms/step - accuracy: 0.1658 - loss: 2.2056 - val_accuracy: 0.2617 - val_loss: 1.9848
Epoch 2/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 82s 784ms/step - accuracy: 0.2992 - loss: 1.9373 - val_accuracy: 0.3533 - val_loss: 1.8247
Epoch 3/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 81s 776ms/step - accuracy: 0.4116 - loss: 1.6653 - val_accuracy: 0.4183 - val_loss: 1.6874
Epoch 4/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 88s 859ms/step - accuracy: 0.5301 - loss: 1.3916 - val_accuracy: 0.4633 - val_loss: 1.6291
Epoch 5/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 75s 768ms/step - accuracy: 0.6104 - loss: 1.1148 - val_accuracy: 0.5000 - val_loss: 1.5417
Epoch 6/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 58s 768ms/step - accuracy: 0.6896 - loss: 0.9032 - val_accuracy: 0.5717 - val_loss: 1.3785
Epoch 7/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 81s 766ms/step - accuracy: 0.7622 - loss: 0.7107 - val_accuracy: 0.5583 - val_loss: 1.4835
Epoch 8/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 80s 738ms/step - accuracy: 0.7954 - loss: 0.6221 - val_accu

In [13]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Precisión del modelo: {acc*100:.2f}%")


19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 169ms/step - accuracy: 0.6085 - loss: 2.4920
Precisión del modelo: 61.67%


El modelo logró una precisión entre 62% en el conjunto de prueba.

La CNN fue capaz de extraer características espaciales relevantes sin embargo no tiene la mejor presión.